In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (12, 8)
plt.rcParams["font.size"] = 12

In [ ]:
import xarray as xr

import xarray_sentinel

In [ ]:
# uncomment download the data
#! DHUS_USER=<user> DHUS_PASSWORD=<password> ./download_data.sh

In [ ]:
product_path = (
    "data/S1B_S6_GRDH_1SDV_20211216T115438_20211216T115501_030050_03968A_0F8A.SAFE"
)
swath_group = "S6"
swath_polarisation_group = "S6/VV"
measurement_group = "S6/VV"
measurement_block_slices = (slice(4096, 6144), slice(None, 2048))
digital_number_max = 150

!ls -d {product_path}

In [ ]:
xr.open_dataset(product_path, engine="sentinel-1")

In [ ]:
xr.open_dataset(product_path, engine="sentinel-1", group=swath_group)

In [ ]:
xr.open_dataset(product_path, engine="sentinel-1", group=swath_polarisation_group)

In [ ]:
measurement = xr.open_dataset(
    product_path, engine="sentinel-1", group=measurement_group, chunks=2048
)
measurement

In [ ]:
measurement_block = measurement.measurement[measurement_block_slices]
measurement_block

In [ ]:
_ = abs(measurement_block).plot(y="azimuth_time", vmax=digital_number_max)

In [ ]:
gcp = xr.open_dataset(
    product_path, engine="sentinel-1", group=f"{swath_polarisation_group}/gcp"
)
gcp

In [ ]:
_ = gcp.height.plot(y="azimuth_time")

In [ ]:
_ = gcp.plot.scatter(x="longitude", y="latitude", hue="height")

In [ ]:
orbit = xr.open_dataset(
    product_path, engine="sentinel-1", group=f"{swath_polarisation_group}/orbit"
)
orbit

In [ ]:
orbit.plot.scatter(y="azimuth_time", x="position", hue="velocity")

In [ ]:
calibration = xr.open_dataset(
    product_path, engine="sentinel-1", group=f"{swath_polarisation_group}/calibration"
)
calibration

In [ ]:
# betaNought and dn calibration are typically constat
print(calibration.betaNought.mean().item(), "+-", calibration.betaNought.std().item())
print(calibration.dn.mean().item(), "+-", calibration.dn.std().item())

In [ ]:
_ = calibration.sigmaNought.plot(x="pixel")

In [ ]:
_ = calibration.gamma.plot(x="pixel")

In [ ]:
betaNought_block = xarray_sentinel.calibrate_amplitude(
    measurement_block, calibration.betaNought
)
betaNought_block

In [ ]:
_ = abs(betaNought_block).plot(y="azimuth_time", vmax=1)

In [ ]:
betaNought_block_db = xarray_sentinel.calibrate_intensity(
    measurement_block,
    calibration.betaNought,
    as_db=True,
)
betaNought_block_db

In [ ]:
_ = betaNought_block_db.plot(y="azimuth_time", vmin=-20, vmax=5)

In [ ]:
noise_range = xr.open_dataset(
    product_path, engine="sentinel-1", group=f"{swath_polarisation_group}/noise_range"
)
noise_range

In [ ]:
_ = noise_range.noiseRangeLut.plot(x="pixel")

In [ ]:
noise_azimuth = xr.open_dataset(
    product_path, engine="sentinel-1", group=f"{swath_polarisation_group}/noise_azimuth"
)
noise_azimuth

In [ ]:
# _ = noise_azimuth.plot.scatter(x="line", y="noiseAzimuthLut")

In [ ]:
coordinate_conversion = xr.open_dataset(
    product_path,
    engine="sentinel-1",
    group=f"{swath_polarisation_group}/coordinate_conversion",
)
coordinate_conversion

In [ ]:
xr.open_dataset(
    product_path, engine="sentinel-1", group=f"{swath_polarisation_group}/attitude"
)

In [ ]:
xr.open_dataset(
    product_path, engine="sentinel-1", group=f"{swath_polarisation_group}/dc_estimate"
)

In [ ]:
xr.open_dataset(
    product_path,
    engine="sentinel-1",
    group=f"{swath_polarisation_group}/azimuth_fm_rate",
)